# Draft Pipeline

## Set Up

In [1]:
# load libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# load data
raw_df = pd.read_csv('../../data/raw/rawdata_new.csv')

# create copy of raw data
processed_df = raw_df.copy()

## Data Prep

### Add predictors

In [2]:
# environment minded
environment_minded_cols = ['GHG_goal', 'env_goal', 'biodiv_goal']
processed_df['environment_minded'] = processed_df[environment_minded_cols].mean(axis=1)

In [3]:
# perceive weather changes
perceive_weather_cols = ['percep_hail', 'percep_drought', 'percep_frost', 'percep_heavyrain', 'percep_longrain', 'percep_heat']
processed_df['perceive_weather_change'] = processed_df[perceive_weather_cols].mean(axis=1)

In [4]:
# anticipate negative consequences
anticipate_negative_cols = ['cons_general', 'cons_farm']
processed_df['anticipate_negative_consequences'] = processed_df[anticipate_negative_cols].mean(axis=1)

In [5]:
# capable implement
capable_cols = ['self_reduc', 'self_act', 'self_cap', 'self_conf', 'self_not']
processed_df['capable_implement'] = processed_df[capable_cols].mean(axis=1)

In [6]:
# social connectedness
social_cols = ['net_name1_imp', 'net_name2_imp', 'net_name3_imp', 'net_name4_imp', 'net_name5_imp', 'net_name6_imp', 'net_name7_imp', 'net_name8_imp', 'net_name9_imp', 'net_name10_imp']
processed_df['social_connectedness'] = processed_df[social_cols].mean(axis=1)

In [7]:
# think measures effective

# create columns that include effectiveness ratings only for applicable mitigations
measure_cols = ['legum', 'conc', 'add', 'lact', 'breed', 'covman', 'comp', 'drag', 'cov', 'plough', 'solar', 'biog', 'ecodr']
for measure in measure_cols:
    measure_eff = measure + '_eff' # effective
    measure_app = measure + '_app' # applicable
    measure_eff_app = measure + '_eff_app' # effective and applicable
    processed_df[measure_app] = [np.NAN if x == 3 else 1 for x in processed_df[measure]]
    processed_df[measure_eff_app] = processed_df[measure_eff] * processed_df[measure_app]

# take average of effectiveness ratings
measures_eff_app_cols = [c + '_eff_app' for c in measure_cols]
processed_df['think_measures_effective'] = processed_df[measures_eff_app_cols].mean(axis=1)

# find record with NAN, change to NA to average rating across farms
nan_row = np.where(processed_df['think_measures_effective'].isna())[0][0]
nan_col = np.where(processed_df.columns == 'think_measures_effective')[0][0]
processed_df.iloc[nan_row, nan_col] = processed_df['think_measures_effective'].mean()

### Add outcome variable

In [8]:
# measures implemented

# calculate proportion of applicable measures implemented
implemented_cols = ['legum', 'conc', 'add', 'lact', 'breed', 'covman', 'comp', 'drag', 'cov', 'plough', 'solar', 'biog', 'ecodr']
processed_df['num_implemented'] = (processed_df[implemented_cols] == 1).sum(axis=1)
processed_df['num_applicable'] = (processed_df[implemented_cols] != 3).sum(axis=1)
processed_df['prop_implemented'] = processed_df['num_implemented'] / processed_df['num_applicable']

## Model Fit

In [9]:
Y = processed_df['prop_implemented']
X = processed_df[['environment_minded', 'perceive_weather_change', 'anticipate_negative_consequences', 'capable_implement', 'think_measures_effective', 'social_connectedness']]
X = sm.add_constant(X)
model = sm.OLS(Y, X)
results = model.fit()

## Metrics and Visualizations

In [12]:
# view summary table
Stargazer([results])